In [ ]:
pip install langchain langgraph groq tiktoken tiktoken langchain-groq langchainhub chromadb langchain_huggingface  langchain langgraph cassio langchain_community google-auth-httplib2 google-auth-oauthlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2 MB/s eta 0:00:

In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field
from google.colab import userdata

from langchain_groq import ChatGroq
import os

from langchain.schema import HumanMessage, SystemMessage
import os
from datetime import datetime, timedelta
import pytz
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from typing_extensions import TypedDict
from typing import Optional, List, Any
from langgraph.graph import StateGraph, END




/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")


Interview Agent

In [ ]:
class InterviewAgent:
    def __init__(self, llm):
        self.llm = llm
        self.collected_data = {}

    def ask(self, question):
        print(f"\n Question : {question}")
        return input(" Ta réponse : ")

    def run(self):
        print("Bienvenue  Je vais t’aider à créer un plan d’apprentissage personnalisé ")

        self.collected_data["goal"] = self.ask("Qu’est-ce que tu veux apprendre ?")
        self.collected_data["weekly_hours"] = int(self.ask("Combien d’heures veux-tu y consacrer par semaine ?"))
        self.collected_data["preferred_days"] = self.ask("Quels jours préfères-tu apprendre ? (ex: lundi, mardi)").replace(" ", "").split(",")
        self.collected_data["preferred_times"] = self.ask("Quelles plages horaires préfères-tu ? (ex: 18:00-20:00)").replace(" ", "").split(",")
        self.collected_data["excluded_days"] = self.ask("Y a-t-il des jours à éviter ? (laisser vide si aucun)").replace(" ", "").split(",")
        self.collected_data["planning_weeks"] = int(self.ask("Sur combien de semaines veux-tu répartir ton apprentissage ?"))


        print("\n Résumé de tes préférences :")
        for k, v in self.collected_data.items():
            print(f" - {k}: {v}")

        return self.collected_data




# === TEST ===
#agent = InterviewAgent(llm)
#user_info = agent.run()
#print("\n Données structurées :", user_info)


Refine Agent

In [ ]:
def refine_user_input_with_llm(user_data, llm):
    instruction = """
Tu es un assistant de structuration de préférences utilisateur.

Ta tâche est de convertir un dictionnaire utilisateur flou en un dictionnaire Python valide, structuré, et interprété proprement.
Tu dois interpréter les valeurs comme :
- 'tous' ou 'tous les jours' → ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche']
- 'tous' ou 'le matin' → ['08:00-12:00']
- 'l'après-midi' → ['14:00-18:00']
- 'non', 'aucun', 'nnon' → []

Tu dois retourner uniquement un dictionnaire Python brut (sans ```), utilisant des guillemets simples dans les chaînes.
N'inclus aucun texte explicatif ni commentaire.
"""

    messages = [
        SystemMessage(content=instruction),
        HumanMessage(content=f"{user_data}")
    ]

    response = llm.invoke(messages)
    raw = response.content.strip()

    #  Nettoyage des blocs markdown ```python
    if raw.startswith("```"):
        raw = raw.replace("```python", "").replace("```", "").strip()

    try:
        parsed = eval(raw)


        if "preferred_days" not in parsed or not parsed["preferred_days"]:
            parsed["preferred_days"] = ["lundi", "mardi", "mercredi", "jeudi", "vendredi"]
        if "preferred_times" not in parsed or not parsed["preferred_times"]:
            parsed["preferred_times"] = ["08:00-12:00", "14:00-18:00"]
        if "excluded_days" not in parsed or not isinstance(parsed["excluded_days"], list):
            parsed["excluded_days"] = []

        return parsed

    except Exception as e:
        print(" Erreur de parsing. Réponse brute :\n", raw)
        raise e




Roadmap Agent

In [ ]:
def generate_learning_roadmap(user_data, llm):
    instruction =  f"""
Tu es un expert en pédagogie assistée par intelligence artificielle.

À partir des informations suivantes :
- Objectif d'apprentissage : {user_data['goal']}
- Charge de travail hebdomadaire souhaitée : {user_data['weekly_hours']} heures
- Durée totale du programme : {user_data['planning_weeks']} semaines

Ta mission est de concevoir une **roadmap d’apprentissage structurée**, découpée en modules progressifs, adaptée à un rythme réaliste et à un niveau débutant à intermédiaire (sauf indication contraire dans l'objectif).

Le résultat attendu est une **liste JSON valide**, contenant 1 dictionnaire par module. Chaque module doit inclure les éléments suivants :
- module : titre clair du module (string)
- duration_h : durée approximative du module en heures (entier ≥ 1)
- objectives : liste de 2 à 4 objectifs d’apprentissage précis et mesurables (liste de strings)
- resources : 2 à 4 ressources fiables et accessibles en ligne (liens de cours, vidéos, plateformes, documents)

 Contraintes :
- La somme des `duration_h` de tous les modules doit représenter environ `{user_data['weekly_hours'] * user_data['planning_weeks']}` heures au total.
- Les ressources doivent être réalistes et correspondre aux objectifs du module.
- Utilise **uniquement des guillemets simples** à l’intérieur des strings JSON pour éviter les erreurs de format.
- Ne génère aucun texte avant ou après la liste. Ne mets pas de ```json ou de markdown.

Ta sortie doit être **exclusivement** une liste JSON de dictionnaires bien formés, sans texte explicatif ni commentaires. Le résultat sera directement interprété par un parser.
"""


    messages = [SystemMessage(content=instruction)]
    response = llm.invoke(messages)

    try:
        import json
        roadmap = json.loads(response.content)
        return roadmap
    except:
        print(" Erreur dans le parsing JSON. Réponse brute :\n", response.content)
        raise


Linking Google Calendar

In [ ]:

def google_calendar_service():
    scopes = ["https://www.googleapis.com/auth/calendar"]
    flow = InstalledAppFlow.from_client_secrets_file(
        "client_secret.json",
        scopes=scopes,
        redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # Add this line
    )

    auth_url, _ = flow.authorization_url(prompt='consent')
    print("Please go to this URL in your browser: ", auth_url)
    print("Copy the authorization code and paste it below.")

    code = input("Enter the authorization code: ")
    flow.fetch_token(code=code)  # Exchange authorization code for access token

    credentials = flow.credentials
    service = build("calendar", "v3", credentials=credentials)
    return service

service = google_calendar_service()

Please go to this URL in your browser:  https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=479794375005-r09q4p7ioivb077631a0p2hqg6rdia46.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=ZPQ2I4KqvyOFUaexiugT1WE7CitZ5m&prompt=consent&access_type=offline
Copy the authorization code and paste it below.
Enter the authorization code: 4/1AUJR-x6leU788_fZN7z4NC9WXRh1Hts4eUYVcc9F0Wjo-B-SxaWro5x0AAI


Fixing time slots :

In [ ]:
def is_slot_available(service, calendar_id, date, start_time, end_time, time_zone='Europe/Paris'):
    tz = pytz.timezone(time_zone)

    start_dt = tz.localize(datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M"))
    end_dt = tz.localize(datetime.strptime(f"{date} {end_time}", "%Y-%m-%d %H:%M"))

    events_result = service.events().list(
        calendarId=calendar_id,
        timeMin=start_dt.isoformat(),
        timeMax=end_dt.isoformat(),
        singleEvents=True,
        orderBy="startTime"
    ).execute()

    events = events_result.get("items", [])
    return len(events) == 0

def schedule_learning_sessions_with_availability_check(roadmap, user_data, service, calendar_id="primary", start_date=None):
    if start_date is None:
        start_date = datetime.today()
    planning_weeks = user_data.get("planning_weeks", 3)


    end_date = start_date + timedelta(weeks=planning_weeks)

    days_map = {
        "lundi": 0, "mardi": 1, "mercredi": 2, "jeudi": 3,
        "vendredi": 4, "samedi": 5, "dimanche": 6
    }

    preferred_days_num = [days_map[d.lower()] for d in user_data["preferred_days"] if d.lower() in days_map]
    excluded_dates = set(user_data["excluded_days"])
    preferred_times = user_data["preferred_times"]

    scheduled = []
    current_date = start_date

    for module in roadmap:
        remaining_hours = module["duration_h"]
        while remaining_hours > 0 and current_date <= end_date:

            if current_date.weekday() in preferred_days_num:
                iso_date = current_date.strftime("%Y-%m-%d")
                if iso_date not in excluded_dates:
                    for time_slot in preferred_times:
                        start_str, end_str = time_slot.split("-")
                        start_hour = int(start_str.split(":")[0])
                        end_hour = int(end_str.split(":")[0])
                        slot_hours = end_hour - start_hour
                        if slot_hours == 0:
                            continue

                        duration = min(remaining_hours, slot_hours)
                        s_time = f"{start_hour:02}:00"
                        e_time = f"{start_hour + duration:02}:00"

                        #  Vérifier la disponibilité
                        if is_slot_available(service, calendar_id, iso_date, s_time, e_time):
                            session = {
                                "module": module["module"],
                                "date": iso_date,
                                "start_time": s_time,
                                "end_time": e_time,
                                "objectives": module["objectives"],
                                "resources": module["resources"]
                            }
                            scheduled.append(session)
                            remaining_hours -= duration
                            break
            current_date += timedelta(days=1)
    return scheduled



Scheduling

In [ ]:
def push_schedule_to_calendar(learning_schedule, service, calendar_id="primary"):
    for session in learning_schedule :
        from datetime import datetime

        start_datetime = datetime.strptime(f"{session['date']} {session['start_time']}", "%Y-%m-%d %H:%M")
        end_datetime = datetime.strptime(f"{session['date']} {session['end_time']}", "%Y-%m-%d %H:%M")

        event = {
            'summary': session['module'],
            'description': " Objectifs : " + ", ".join(session["objectives"]) + "\n" +
                           " Ressources : " + "\n".join(session["resources"]),
            'start': {
                'dateTime': start_datetime.isoformat(),
                'timeZone': 'Europe/Paris',
            },
            'end': {
                'dateTime': end_datetime.isoformat(),
                'timeZone': 'Europe/Paris',
            },
        }

        created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
        print(f" Événement ajouté : {created_event.get('htmlLink')}")


Langraph Graphstate

In [ ]:


class GraphState(TypedDict):
    user_raw_input: Optional[dict]
    user_refined_input: Optional[dict]
    roadmap: Optional[list]
    schedule: Optional[list]
    calendar_pushed: Optional[bool]
    calendar_service: Any
    user_approved_calendar: Optional[bool]




Defining the nodes

In [ ]:
def interview_node(state):
    state["user_raw_input"] = InterviewAgent(llm).run()
    return state

def refine_node(state):
    state["user_refined_input"] = refine_user_input_with_llm(state["user_raw_input"], llm)
    return state

def plan_node(state):
    state["roadmap"] = generate_learning_roadmap(state["user_refined_input"], llm)
    return state

def schedule_node(state):
    from pprint import pprint
    roadmap = state["roadmap"]
    user_data = state["user_refined_input"]
    service = state["calendar_service"]

    state["schedule"] = schedule_learning_sessions_with_availability_check(
        roadmap,
        user_data,
        service=service,
        calendar_id="primary"
    )
    pprint(state["schedule"])
    return state

def validation_node(state):
    from pprint import pprint

    print("\n Roadmap :")
    for module in state["roadmap"]:
        print(f"- {module['module']} ({module['duration_h']}h)")

    print("\n Planning :")
    for session in state["schedule"]:
        print(f"{session['date']} | {session['start_time']} → {session['end_time']} | {session['module']}")

    response = input("\n Souhaites-tu ajouter ce planning dans Google Calendar ? (oui/non) : ").strip().lower()
    approved = response in ["oui", "yes", "y"]

    state["user_approved_calendar"] = approved
    if approved:
        print(" Confirmation : le planning sera ajouté dans Google Calendar.")
    else:
        print(" Le planning ne sera pas ajouté dans Google Calendar.")

    return state


def calendar_node(state):
    if not state.get("user_approved_calendar", False):
        print(" L'utilisateur a refusé l'ajout dans Google Calendar.")
        return state


    service = state["calendar_service"]
    push_schedule_to_calendar(state["schedule"], service)
    state["calendar_pushed"] = True
    print("Planning ajouté à Google Calendar.")
    return state






Building the graph

In [ ]:
builder = StateGraph(GraphState)

builder.add_node("interview", interview_node)
builder.add_node("refine", refine_node)
builder.add_node("plan", plan_node)
builder.add_node("schedule_agent", schedule_node)
builder.add_node("calendar", calendar_node)
builder.add_node("validate", validation_node)

builder.set_entry_point("interview")
builder.add_edge("interview", "refine")
builder.add_edge("refine", "plan")
builder.add_edge("plan", "schedule_agent")
builder.add_edge("schedule_agent", "validate")
builder.add_edge("validate", "calendar")
builder.add_edge("calendar", END)



initial_state = {
    "user_raw_input": None,
    "user_refined_input": None,
    "roadmap": None,
    "schedule": None,
    "calendar_pushed": False,
    "calendar_service": service,
    "user_approved_calendar": False
}





In [ ]:
graph = builder.compile()

In [ ]:
print("Clés dans initial_state :", list(initial_state.keys()))


Clés dans initial_state : ['user_raw_input', 'user_refined_input', 'roadmap', 'schedule', 'calendar_pushed', 'calendar_service', 'user_approved_calendar']


Asking for Roadmap and setting the plan on calendar :

In [ ]:
final_state = graph.invoke(initial_state)

Bienvenue  Je vais t’aider à créer un plan d’apprentissage personnalisé 

 Question : Qu’est-ce que tu veux apprendre ?
 Ta réponse : chinois

 Question : Combien d’heures veux-tu y consacrer par semaine ?
 Ta réponse : 2

 Question : Quels jours préfères-tu apprendre ? (ex: lundi, mardi)
 Ta réponse : tous

 Question : Quelles plages horaires préfères-tu ? (ex: 18:00-20:00)
 Ta réponse : tous

 Question : Y a-t-il des jours à éviter ? (laisser vide si aucun)
 Ta réponse : non

 Question : Sur combien de semaines veux-tu répartir ton apprentissage ?
 Ta réponse : 2

 Résumé de tes préférences :
 - goal: chinois
 - weekly_hours: 2
 - preferred_days: ['tous']
 - preferred_times: ['tous']
 - excluded_days: ['non']
 - planning_weeks: 2
[{'date': '2025-05-15',
  'end_time': '09:00',
  'module': 'Introduction au chinois',
  'objectives': ["Comprendre les bases de l'alphabet chinois (hanzi).",
                 'Apprendre les salutations et les phrases de base en chinois.',
                 'I

In [ ]:
from pprint import pprint
pprint(final_state)


{'calendar_pushed': True,
 'calendar_service': <googleapiclient.discovery.Resource object at 0x7fdc5eacd150>,
 'roadmap': [{'duration_h': 1,
              'module': 'Introduction au chinois',
              'objectives': ["Comprendre les bases de l'alphabet chinois "
                             '(hanzi).',
                             'Apprendre les salutations et les phrases de base '
                             'en chinois.',
                             'Identifier les tons chinois et leur importance.'],
              'resources': ['https://www.chinesepod.com/learn-chinese/beginner',
                            'https://www.youtube.com/watch?v=9Y2yO7R26qg']},
             {'duration_h': 1,
              'module': 'Grammaire essentielle',
              'objectives': ['Maîtriser les structures grammaticales simples '
                             '(sujet-verbe-objet).',
                             'Utiliser les particules chinoises de base.',
                             'Construire 